# Model Evaluation Proposed by Hari Thapliyal

In [2]:
import pandas as pd
import numpy as np


In [ ]:
samples=20000
y=np.random.binomial(1,.2,samples) # Ground Truth
ypred1 = np.random.binomial(1,.21,samples)
ypred2 = np.random.binomial(1,.24,samples)
ypred3 = np.random.binomial(1,.28,samples)
ypred4 = np.random.binomial(1,.32,samples)
ypred5 = np.random.binomial(1,.35,samples)

In [169]:
samples=200000
y=np.random.binomial(1,.020,samples) # Ground Truth
ypred1 = np.random.binomial(1,.021,samples)
ypred2 = np.random.binomial(1,.025,samples)
ypred3 = np.random.binomial(1,.030,samples)
ypred4 = np.random.binomial(1,.032,samples)
ypred5 = np.random.binomial(1,.037,samples)


df = pd.DataFrame({"y":y, "ypred1":ypred1, "ypred2":ypred2, "ypred3": ypred3, "ypred4": ypred4, "ypred5":ypred5})


In [170]:
import sklearn.metrics as metrics

metrics.confusion_matrix(df.y,df.ypred1)

array([[191695,   4187],
       [  4034,     84]])

In [171]:
metrics.confusion_matrix(df.y,df.ypred2)

array([[191046,   4836],
       [  4011,    107]])

In [172]:
model1_res = np.array(metrics.confusion_matrix(df.y,df.ypred1)).reshape(-1)
model2_res = np.array(metrics.confusion_matrix(df.y,df.ypred2)).reshape(-1)
model3_res = np.array(metrics.confusion_matrix(df.y,df.ypred3)).reshape(-1)
model4_res = np.array(metrics.confusion_matrix(df.y,df.ypred4)).reshape(-1)
model5_res = np.array(metrics.confusion_matrix(df.y,df.ypred5)).reshape(-1)
df2 = pd.DataFrame({"model1_res": model1_res, "model2_res": model2_res, "model3_res": model3_res, "model4_res": model4_res, "model5_res":model5_res}, index=['TN','FP','FN','TP'])
df2=pd.DataFrame.transpose(df2)
df2

,TN,FP,FN,TP
model1_res,191695,4187,4034,84
model2_res,191046,4836,4011,107
model3_res,189925,5957,3997,121
model4_res,189619,6263,3994,124
model5_res,188691,7191,3954,164


In [173]:
df2['Pos_Pred'] = [len(df[df.ypred1==1]), len(df[df.ypred2==1]), len(df[df.ypred3==1]),len(df[df.ypred4==1]),len(df[df.ypred5==1])]

In [174]:
df2['Acc'] = (df2.TP + df2.TN)/ len(df)
df2['Precision'] = (df2.TP)/ df2.Pos_Pred
df2['Recall'] = (df2.TP )/ len(df[df.y==1])
df2['FScore'] = 2 * (df2.Precision * df2.Recall)/ (df2.Precision + df2.Recall)


In [175]:
df.y.value_counts() # Unbalanced Dataset

0    195882
1      4118
Name: y, dtype: int64

In [176]:
df2

,TN,FP,FN,TP,Pos_Pred,Acc,Precision,Recall,FScore
model1_res,191695,4187,4034,84,4271,0.958895,0.019668,0.020398,0.020026
model2_res,191046,4836,4011,107,4943,0.955765,0.021647,0.025983,0.023618
model3_res,189925,5957,3997,121,6078,0.950230,0.019908,0.029383,0.023735
model4_res,189619,6263,3994,124,6387,0.948715,0.019414,0.030112,0.023608
model5_res,188691,7191,3954,164,7355,0.944275,0.022298,0.039825,0.028589


- If we pick Accuracy metrics to select model then Model1 will be selected.
- If we pick Precision metrics to select model then Model1 will be selected.
- If we pick Recall metrics to select model then Model5 will be selected.
- If we pick F1Score metrics to select model then Model4 will be selected.

In [177]:
# Lets define the business objective in terms of profit.
# - means loss. Every TP fetches $1000, Every TN saves $20, Every FP leads to loss of 100, Every FN leads to -80. FN is less expensive compare to FP

Business_Profit = {"TP": 1000 , "TN":20, "FP":-100, "FN":-2000}

In [178]:
df_business = pd.DataFrame()
for col in ['TP','TN','FP','FN']:
  df_business[col] = Business_Profit[col]*df2[col]

df_business

,TP,TN,FP,FN
model1_res,84000,3833900,-418700,-8068000
model2_res,107000,3820920,-483600,-8022000
model3_res,121000,3798500,-595700,-7994000
model4_res,124000,3792380,-626300,-7988000
model5_res,164000,3773820,-719100,-7908000


In [179]:
df2['Business_Profit'] = df_business.sum(axis=1)
df2

,TN,FP,FN,TP,Pos_Pred,Acc,Precision,Recall,FScore,Business_Profit
model1_res,191695,4187,4034,84,4271,0.958895,0.019668,0.020398,0.020026,-4568800
model2_res,191046,4836,4011,107,4943,0.955765,0.021647,0.025983,0.023618,-4577680
model3_res,189925,5957,3997,121,6078,0.950230,0.019908,0.029383,0.023735,-4670200
model4_res,189619,6263,3994,124,6387,0.948715,0.019414,0.030112,0.023608,-4697920
model5_res,188691,7191,3954,164,7355,0.944275,0.022298,0.039825,0.028589,-4689280


In [180]:
# Based on the give business objective. Model1 makes most profit therefore we will select model1.
# If dataset is imbalance then we choose recall or precision or f1 score metric to evaluate the model. \
# If you choose those metrics then model seleced are different.

# Whether dataset is balance or imbalance if choose Business Objective then we can select correct model.